In [1]:
#### 双向最大匹配分词方法实现
class BiMM():
    def __init__(self, user_dict):
        self.user_dict = user_dict
    def cut(self, sentence):
        """
        正向最大匹配（FMM）
        :param user_dict: 词典
        :param sentence: 句子
        """
        # 词典中最长词长度
        max_len = max([len(item) for item in self.user_dict])
        start = 0
        f_single_word_num = 0 #单字词数量计数
        f_segs = [] #存储分词结果
        while start != len(sentence):
            index = start+max_len
            if index>len(sentence):
                index = len(sentence)
            for i in range(max_len):
                if (sentence[start:index] in self.user_dict) or (len(sentence[start:index])==1):
                    f_segs.append(sentence[start:index])
                    if index-start==1:
                        f_single_word_num+=1
                    #print(sentence[start:index], end='/')
                    start = index
                    break
                index += -1
        """
        反向最大匹配（BMM）
        :param user_dict:词典
        :param sentence:句子
        """
        result = [] #暂存分词结果
        b_segs = [] #存储分词结果
        b_single_word_num = 0 #单字词数量计数
        start = len(sentence)
        while start != 0:
            index = start - max_len
            if index < 0:
                index = 0
            for i in range(max_len):
                #print(sentence[index:start])
                if (sentence[index:start] in self.user_dict) or (len(sentence[index:start])==1):
                    result.append(sentence[index:start])
                    if start-index == 1:
                        b_single_word_num+=1
                    start = index
                    break
                index += 1
        b_segs = result[::-1]
    
        """
        双向最大匹配
        """
        if len(f_segs)<len(b_segs):
            return f_segs
        elif len(f_segs)>len(b_segs):
            return b_segs
        else:
            if f_single_word_num>b_single_word_num:
                return b_segs
            else:
                return f_segs

In [2]:
#测试双向最大匹配分词方法
user_dict = ['我们','在', '在野', '生动', '野生', '动物园', '野生动物园', '物','园','玩']
sentence = '我们在野生动物园玩'
bimm = BiMM(user_dict)
print(bimm.cut(sentence))

['我们', '在', '野生动物园', '玩']


In [20]:
#从人民日报分词语料中读取分词数据，并构建词典
#将人民日报分词语料还原成原文
ribao_dict = [] #语料中出现的所有词
ribao_dict_sorted = [] #过滤掉低频词后的词典
yuanwen = [] #存储语料原文
punctuation = []  #存储标点符号
c = 0
with open(r"C:\Users\Administrator.DESKTOP-TSVP15E\Downloads\199801 (2).txt","r",encoding="utf-8") as fr, \
open(r"C:\Users\Administrator.DESKTOP-TSVP15E\Downloads\199801_source.txt","w",encoding="utf-8") as fw:
    for line in fr:
        #if c > 100:
        #    break
        ls = line.strip().split('  ')
        line_segs = []
        for i in range(1,len(ls)):
            #print(ls[i])
            #if '/' in ls[i]:
            end_idx = ls[i].index('/')
            line_segs.append(ls[i][:end_idx])
            ribao_dict.append(ls[i][:end_idx])
        
            if ls[i].endswith('/w'):
                punctuation.append(ls[i][:end_idx])
        fw.write(''.join(line_segs)+'\n')
        yuanwen.append(''.join(line_segs))
        #c+=1
word_count_dict = {}
for word in ribao_dict:
    if word not in word_count_dict:
        word_count_dict[word]=1
    else:
        word_count_dict[word]+=1
sorted_word_count = sorted(word_count_dict.items(), key = lambda item:item[1], reverse=True)

#保存出现频次大于2的词，存入词典
for k,v in sorted_word_count:
    if v>2:
        ribao_dict_sorted.append(k)
ribao_dict_set = set(ribao_dict)
punctuation = set(punctuation)
print(len(ribao_dict_set),len(punctuation), len(ribao_dict_sorted))

56482 47 22111


In [21]:
#使用新词典测试双向最大匹配分词方法
bimm = BiMM(ribao_dict_sorted)
bimm.cut("迈向新世纪")

['迈向', '新', '世纪']

In [23]:
import time
from tqdm import tqdm #进度条线上模块
time_start = time.time() #计时
source_text = []
with open(r"C:\Users\Administrator.DESKTOP-TSVP15E\Downloads\199801_source.txt","r",encoding="utf-8") as fr:
    for line in fr:
        source_text.append(line.strip())

bimm = BiMM(ribao_dict_sorted)
with open(r"C:\Users\Administrator.DESKTOP-TSVP15E\Downloads\199801_source_segs.txt","w",encoding="utf-8") as fw:
    for t in tqdm(range(len(source_text)),desc = 'segging'):
        text = source_text[t]  
        start_idx = 0
        Bi_segs = []
        flag = True
        for idx in range(len(text)):
            if text[idx] in punctuation: #有标点符号的，先根据标点符号节分句子
                Bi_segs.extend(bimm.cut(text[start_idx:idx+1]))
                start_idx = idx+1
                flag = False
        if flag:
            Bi_segs.extend(bimm.cut(text))
        fw.write('/  '.join(Bi_segs)+'\n')
time_end = time.time()
print(time_end-time_start)

segging: 100%|█████████████████████████████████████████████████████████████████| 23064/23064 [5:50:12<00:00,  1.10it/s]

21012.682418107986


In [39]:
# ... 其他函数 ...

def generate_text(trigrams, bigrams, start_word, length=100):
    sentence = [start_word, None]  # 初始化句子为包含至少两个元素的列表
    while len(sentence) < length:
        bigram = (sentence[-2], sentence[-1])
        if bigram not in trigrams or not trigrams[bigram]:
            # 如果没有更多的TriGram或Bigram，则停止生成
            # 同时检查是否已经生成了句子结束符
            if sentence[-1] == '.':
                break
            # 如果句子不完整，尝试添加句子结束符并停止生成
            sentence.append('.')
            break
        
        # 根据概率选择下一个词
        next_word, count = random.choices(list(trigrams[bigram].keys()), weights=trigrams[bigram].values())[0]
        sentence.append(next_word)
        
        # 如果生成了句子结束符，也停止生成
        if next_word == '.':
            break
    return ' '.join(sentence)

def main():
    file_path = "C:\\Users\\Administrator.DESKTOP-TSVP15E\\Downloads\\199801_source.txt"
    text = read_data(file_path)
    words = preprocess_text(text)
    bigrams, trigrams = build_trigram_model(words)
    
    # 选择一个随机的BiGram作为起始点，并确保起始词是bigram的第一个词
    start_bigram = random.choice(list(bigrams.keys()))
    start_word = start_bigram[0]  # 确保start_word是bigram的第一个词
    
    # 确保start_word是trigrams中的有效起始词
    while (start_word, start_bigram[1]) not in trigrams or not trigrams[(start_word, start_bigram[1])]:
        start_bigram = random.choice(list(bigrams.keys()))
        start_word = start_bigram[0]
    
    generated_sentence = generate_text(trigrams, bigrams, start_word)
    print(generated_sentence)

# ... 主程序 ...

In [46]:
import os  
from collections import defaultdict, deque  
import random  
  
def read_file(file_path):  
    with open(file_path, 'r', encoding='utf-8') as file:  
        return file.read().split()  # 假设文件已经是分词后的形式  
  
def build_trigram_dict(tokens):  
    trigrams = defaultdict(list)  
    for i in range(len(tokens) - 2):  
        trigram = (tokens[i], tokens[i+1], tokens[i+2])  
        trigrams[trigram[:2]].append(trigram[2])  
    return trigrams  
  
def train_trigram_model(trigrams):  
    model = defaultdict(lambda: defaultdict(int))  
    total_counts = defaultdict(int)  
      
    for (w1, w2), w3s in trigrams.items():  
        for w3 in w3s:  
            model[(w1, w2)][w3] += 1  
            total_counts[(w1, w2)] += 1  
      
    # Normalize probabilities  
    for (w1, w2), count in total_counts.items():  
        for w3, freq in model[(w1, w2)].items():  
            model[(w1, w2)][w3] /= count  
      
    return model  
  
def generate_sentence(model, start_trigram, num_words, unk_token='<UNK>'):  
    sentence = list(start_trigram)  
    history = deque(start_trigram, maxlen=2)  
    word_list = set(tokens)  # 假设tokens是预处理后的所有单词列表  
  
    for _ in range(num_words - 2):  
        next_word_probs = model[tuple(history)]  
        if not next_word_probs:  # 如果字典为空  
            next_word = random.choice(list(word_list))  # 从所有词中随机选择一个  
        else:  
            next_word = random.choices(list(next_word_probs.keys()), list(next_word_probs.values()), k=1)[0]  
        sentence.append(next_word)  
        history.append(next_word)  
      
    return ' '.join(sentence)
  
# 使用示例  
file_path = "C:\\Users\\Administrator.DESKTOP-TSVP15E\\Downloads\\199801_source.txt"  
tokens = read_file(file_path)  
trigrams = build_trigram_dict(tokens)  
model = train_trigram_model(trigrams)  
# 假设我们有一个初始的TriGram  
start_trigram = ()  
generated_sentence = generate_sentence(model, start_trigram, 10)  
print(generated_sentence)

我和老伴都是贫苦出生，都是共产党员，在对待贫困群众的态度上，我们是一致的。我每次下乡时，妻子总是在衣服里装上几百元钱，说是遇事好应急。凡是见到脚上有泥土，穿得不好的乡下人，她都特别热情，从不叫客人脱鞋进屋。客人有什么事，她都用笔认真记下来，贫困群众送的土特产，她怕伤了他们的自尊，一律折价换物。有时因我拿出去接济贫困群众的钱太多，家里生活无法保障，她也从不吭一声，而是先找他人借钱，等下月工资发了再还。说实在的，没有妻子的支持，我先后将家里省吃俭用节约下来的５万多元钱接济给贫困群众是难以做到的。妻子对两个孩子要求也非常严格。这几年孩子大了，懂事了，看到家里比同学家都穷，问妈妈：“我们家的钱到哪去了？”她告诉孩子说：“我们有很多乡下的穷亲戚，都支持他们了。”我们夫妻俩都没有了父母，人民就是我们的父母，孝敬父母是做儿女的本分啊！ 尉健行说，一九九八年是全面贯彻落实党的十五大提出的各项任务的第一年。做好新的一年的党风廉政建设和反腐败工作，对于保证党的十五大确定的目标和任务的顺利实现，具有重要意义。 十一、开庭审判 各乡镇都成立了读报用报活动领导小组，制定了报刊传阅制度，全县５１４个行政村中除建立阅览室外，还在村里人口集中的地方建起了阅报栏，指定专人定期更换。 据新华社瓦莱塔１月１２日电（记者袁锦林）马耳他总统乌戈·米夫萨德·邦尼奇１２日上午在总统府会见了正在这里访问的[中国国务院副总理兼外长钱其琛。 而那乳名，在乡音中生长 近几年来，该院先后投资数千万元，新建了住院大楼，引进了全身ＣＴ、磁共振，１９９６年又引进了血流透析机、平板运动试验及Ｈｏｔｅｒ等心脏检测设备。 李德润葛洪


In [68]:
import jieba  # 假设使用jieba进行中文分词  
import networkx as nx  # 需要在文件顶部导入networkx  
  
def tokenize(text):    
    return list(jieba.cut(text))  
  
def build_graph(words):    
    # 假设 words 是一个分词后的单词列表    
    graph = nx.Graph()    
    # 在这里添加节点和边来构建图  
    # 这里只添加节点作为示例，你可能需要添加边的逻辑  
    for word in words:    
        graph.add_node(word)    
    return graph  
  
# 假设有一个计算PageRank得分的函数，但这里我们只是一个示例，你需要自己实现或找到实现  
def pagerank(graph):  
    # 这里应该使用networkx的pagerank算法来计算节点的PageRank得分  
    # 注意：networkx的pagerank函数需要图作为输入，并返回一个字典，其中键是节点，值是PageRank得分  
    return nx.pagerank(graph)  
  
# 示例用法  
text = "C:\\Users\\Administrator.DESKTOP-TSVP15E\\Desktop\\199801(4).txt"
with open(text, 'r', encoding='utf-8') as f:  # 打开文件并读取内容  
    content = f.read()  
words = tokenize(content)  # 分词  
graph = build_graph(words)  # 构建词图  
scores = pagerank(graph)  # 计算PageRank得分  
  
# 提取得分最高的五个关键词  
top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]  
print("关键词:", [word for word, _ in top_keywords])

关键词: ['19980101', '-', '01', '001', ' ']


In [69]:
import jieba  # 假设使用jieba进行中文分词  
import networkx as nx  # 需要在文件顶部导入networkx  
  
def tokenize(text):    
    return list(jieba.cut(text))  
  
def build_graph(words):    
    # 假设 words 是一个分词后的单词列表    
    graph = nx.Graph()    
    # 在这里添加节点和边来构建图  
    # 这里只添加节点作为示例，你可能需要添加边的逻辑  
    for word in words:    
        graph.add_node(word)    
    return graph  
  
# 假设有一个计算PageRank得分的函数，但这里我们只是一个示例，你需要自己实现或找到实现  
def pagerank(graph):  
    # 这里应该使用networkx的pagerank算法来计算节点的PageRank得分  
    # 注意：networkx的pagerank函数需要图作为输入，并返回一个字典，其中键是节点，值是PageRank得分  
    return nx.pagerank(graph)  
  
# 示例用法  
text = "C:\\Users\\Administrator.DESKTOP-TSVP15E\\Desktop\\199801(5).txt"
with open(text, 'r', encoding='utf-8') as f:  # 打开文件并读取内容  
    content = f.read()  
words = tokenize(content)  # 分词  
graph = build_graph(words)  # 构建词图  
scores = pagerank(graph)  # 计算PageRank得分  
  
# 提取得分最高的五个关键词  
top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]  
print("关键词:", [word for word, _ in top_keywords])

关键词: [' ', '迈向', '充满', '希望', '的']


In [72]:
import jieba  
  
# 读取文件内容  
with open("C:\\Users\\Administrator.DESKTOP-TSVP15E\\Desktop\\199801(5).txt", encoding='utf-8') as f:  
    text = f.read().strip()  
  
# 使用jieba进行分词，并转换为列表  
seg_list = jieba.cut(text, cut_all=False)  
words = list(seg_list)  
  
# 创建一个字典来存储词频  
word_freq = {}  
for word in words:  
    if len(word) > 1:  # 过滤掉单字（可选）  
        if word not in word_freq:  
            word_freq[word] = 1  
        else:  
            word_freq[word] += 1  
  
# 按照词频进行排序  
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  
  
# 假设我们选择前五个词作为关键词  
top_keywords = [word for word, freq in sorted_words[:5]]  
print("关键词:", top_keywords)

关键词: ['中国', '发展', '国际', '人民', '世界']
